## Challenge 1 - Exploring the Data
In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [2]:
import pandas as pd
salaries = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')


In [5]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the head function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [6]:
salaries.isnull().sum()

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below



In [7]:
counts = salaries.groupby('Salary or Hourly').count()['Name']
counts

Salary or Hourly
Hourly     8022
Salary    25161
Name: Name, dtype: int64


What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [9]:
dep = salaries.groupby('Department').count()['Name'].sort_values(ascending=False).reset_index()
dep

,Department,Name
0,POLICE,13414
1,FIRE,4641
2,STREETS & SAN,2198
3,OEMC,2102
4,WATER MGMNT,1879
5,AVIATION,1629
6,TRANSPORTN,1140
7,PUBLIC LIBRARY,1015
8,GENERAL SERVICES,980
9,FAMILY & SUPPORT,615


## Challenge 2 - Hypothesis Tests
In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [10]:
salaries['Hourly Rate'].describe()

count    8022.000000
mean       32.788558
std        12.112573
min         2.650000
25%        21.200000
50%        35.600000
75%        40.200000
max       109.000000
Name: Hourly Rate, dtype: float64

In [11]:
Z= (32.788558 - 30)/(12.112573/(8022)**0.5)
Z

20.61980734597066

In [13]:
from statsmodels.stats.weightstats import ztest as ztest

zscore, pvalue = ztest(salaries[salaries['Hourly Rate'].isnull() == False]['Hourly Rate'], value=30)
print(zscore, pvalue)

20.6198057854942 1.8228873859286195e-94


Since the p value (1.8228873859286195e-94) is lower than alpha (0.05), we reject the null hypothesis (H0) which states that the hourly wage of all hourly workers is not significantly different from 30 dollars per hour. An alternate hypothesis, the one we accept, would state that the hourly wage of all hourly workers is significantly different from 30 dollars per hour.

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [14]:
police = salaries[salaries['Department'] == 'POLICE']
police.groupby('Department').mean()['Annual Salary'].reset_index()

,Department,Annual Salary
0,POLICE,86486.414503


In [15]:
len(police)

13414

In [16]:
zscore, pvalue = ztest(police[police['Annual Salary'].isnull() == False]['Annual Salary'], value=86000)


In [17]:
zscore

3.081997005712994

In [18]:
pvalue

0.002056169057509571

In [19]:
pvalue/2


0.0010280845287547855

Using the crosstab function, find the department that has the most hourly workers.



In [23]:
most_hw = pd.crosstab(index=salaries['Department'], columns=salaries['Salary or Hourly'])
most_hw.sort_values('Hourly', ascending=False)['Hourly'].reset_index()[:1]

,Department,Hourly
0,STREETS & SAN,1862


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [25]:
streetssan = salaries[salaries['Department'] == 'STREETS & SAN']
streetssan.groupby('Department').mean()['Hourly Rate'].reset_index()

,Department,Hourly Rate
0,STREETS & SAN,33.728378


In [27]:
from scipy import stats

t_statistic, p_value = stats.ttest_1samp(a=streetssan[streetssan['Hourly Rate'].isnull() == False]['Hourly Rate'],
                                         popmean=35, alternative='less')
print(t_statistic , p_value)

-9.567447887848152 1.6689265282353859e-21


## Challenge 3 - Constructing Confidence Intervals
While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. ).

In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers.

The confidence interval is computed in SciPy using the t.interval function. You can read more about this function here.

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using this function in SciPy.

In [28]:
mean_hw = salaries[salaries['Salary or Hourly'] == 'Hourly']['Hourly Rate']
mean_hw.mean()

32.78855771628128

In [29]:
stats.t.interval(0.95, df=len(mean_hw)-1, loc=mean_hw.mean(), scale=stats.sem(mean_hw))


(32.52345834488529, 33.05365708767727)

In [30]:

salary_police = salaries[(salaries['Department'] == 'POLICE') & (salaries['Salary or Hourly'] == 'Salary')]

stats.t.interval(0.95, len(salary_police)-1, 
                 loc=salary_police['Annual Salary'].mean(), 
                 scale=stats.sem(salary_police['Annual Salary']))

(86177.05631531784, 86795.77269094894)

## Challenge 4 - Hypothesis Tests of Proportions
Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction.

In the cell below, use the proportions_ztest function from statsmodels to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [32]:
hourly_workers = salaries[salaries['Salary or Hourly'] == 'Hourly']
len(hourly_workers)

8022

In [33]:
from statsmodels.stats.proportion import proportions_ztest

ztest, pval = proportions_ztest(count=len(hourly_workers), nobs=len(salaries),  value=0.25)

In [34]:
print(ztest, pval)

-3.5099964213703005 0.0004481127249057967
